# ESP32 Serial Port

In [15]:
import serial
import time

In [16]:
# Set port to use
port = serial.Serial("/dev/ttyUSB0", 115200, timeout=1)

In [17]:
def command(cmd, data = None):
    dat = bytearray(1)
    dat[0] = cmd
    if data is not None:
        if isinstance(data, bytes):
            dat += data
        else:
            data: int
            dat += data.to_bytes(1, "big")
    dat += b'\x00'
    
    with port as ser:
        ser.write(dat)
        n_bytes = int.from_bytes(ser.read(2), "little")
        n_pack = int.from_bytes(ser.read(2), "little")
        if n_pack == 1:
            yield ser.read(n_bytes)
        else:
            for _ in range(n_pack):
                yield ser.read(n_bytes)

# AD2 Setup

In [18]:
import pydwf
import pydwf.utilities

## Open device

In [20]:
def scorefunc(x):
    return x[pydwf.DwfEnumConfigInfo.AnalogInBufferSize]

dwf = pydwf.DwfLibrary()
dev = pydwf.utilities.openDwfDevice(dwf,
    score_func=scorefunc
)

DwfLibraryError: DWF Library Error 'UnknownError' (1): 'The device is being used by another application SERC: 0x3E9\nDjtgEnable failed.\nDevice programming failed.'

## Setup Device

### Setup Input

In [ ]:
## Settings
inCH = [0, 1]
inFreq = 1e6
inRecordLen = 10e-3
inNSamples = int(inFreq * inRecordLen)

# Get and reset each input channel
anIn = dev.analogIn
anIn.reset()
for CH in inCH:
    anIn.channelEnableSet(CH, True)
    
# Set range and offset for best resolution
anIn.channelRangeSet(0, 30)
anIn.channelOffsetSet(0, 0)
anIn.channelRangeSet(1, 3.5)
anIn.channelOffsetSet(1, 1.65)

# Set acquisition mode and settings
anIn.acquisitionModeSet(pydwf.DwfAcquisitionMode.Record)
anIn.frequencySet(inFreq)
anIn.recordLengthSet(inRecordLen)
anIn.triggerSourceSet(pydwf.DwfTriggerSource.PC)
time.sleep(1)

### Setup Supply

In [ ]:
supplyV = 5

anIO = dev.analogIO
anIO.reset()
anIO.enableSet(True)
anIO.channelNodeSet(0, 0, True) # Enable positive supply
anIO.channelNodeSet(0, 1, supplyV) # Set positive supply voltage
anIO.configure()

## Data reading method

In [ ]:
import numpy as np

In [ ]:
def read_data(nAcq):
    samples = []
    while True:
        status = anIn.status(True)
        if status in [pydwf.DwfState.Armed, pydwf.DwfState.Ready]:
            continue
        aSamples, _, _ = anIn.statusRecord()

        if aSamples != 0:
            cSamples = np.vstack(
                        [anIn.statusData(idx, aSamples)
                         for idx in inCH]).transpose()
            samples.append(cSamples)

        nSamples = sum(len(x) for x in samples)
        print(f"\r\033[K{100*nSamples/nAcq: >4.2f}% {aSamples: >5} {nSamples: >9}", end="")

        if status == pydwf.DwfState.Done and nSamples >= nAcq:
            break

    samples = np.concatenate(samples)[-nAcq:]
    return samples

# Acquire data

In [ ]:
import matplotlib.pyplot as plt
import json
import datetime

## Single measurement and plot

In [ ]:
# Set APESW parameters
list(command(3, b"\x03\x06"))

# Start and trigger AD2
anIn.configure(True, True)
dev.triggerPC()

# Send pulse command to ESP32
list(command(4))

data = read_data(inNSamples)
plt.plot(data)

## Parameter sweeps

In [ ]:
data = []
for n_inv in range(1, 7):
    for n_warm in range(1, 7):
        data_point = {"n_inv": n_inv, "n_warm": n_warm}
        
        # Set APESW parameters
        list(command(3, int.to_bytes(n_inv, 1, "little") + int.to_bytes(n_warm, 1, "little")))
        
        # Start and trigger AD2
        anIn.configure(True, True)
        dev.triggerPC()
        
        # Send pulse command to ESP32
        list(command(4))
        
        data_point["data"] = read_data(inNSamples)
        data.append(data_point)

## Plot parameter sweeps

In [ ]:
plt.close("all")
for dpoint in data:
    fig, ax = plt.subplots()
    ax.plot(dpoint["data"])
    
    plt.title(f"$n_{{inv}} = {dpoint['n_inv']} \quad n_{{warm}} = {dpoint['n_warm']}$")
    # break